In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
class StockPredictKNN:
    def __init__(self, stock_name_, from_, start_date_, end_date_, percent_=0.25):
        self.stock_name = stock_name_
        self.start_date = start_date_
        self.end_date = end_date_
        
        self.success = self.__get_stock_positions(from_)

        if self.success is True:
            self.__split(percent_)
            self.__fit()
            self.__predict()
            
    def __get_stock_positions(self, from_):
        # Download do dataframe.
        try:
            if from_ == 'yahoo':
                self.stock = pdr.get_data_yahoo(self.stock_name,
                                                start=datetime.fromisoformat(self.start_date),
                                                end=datetime.fromisoformat(self.end_date))
            else:
                raise RuntimeError('Lamento, fonte de dados desconhecida.')
        except pdr._utils.RemoteDataError as error:
            print('Oops! Problema na recuperação dos dados!')
            print(error)
            return False
        except:
            print('Lamento, erro genérico!')
            return False
          
        # Eliminando os N/A
        self.stock.dropna()
        
        # Preços no fechamento em relação à abertura, rm R$
        self.prices = self.stock[['Open', 'Close']].values
        
        # Delta no preço
        diff = self.prices[:, 1]-self.prices[:, 0]
        
        # Array de 0s e 1s indicando descidas e subidas, respectivamente.
        self.target = [0 if v < 0 else 1 for v in diff]
        self.prices = StandardScaler().fit_transform(self.prices)
        
        return True
        
    def __split(self, percent_):
        self.prices_train, self.prices_test, \
        self.target_train, self.target_test = train_test_split(
            self.prices, self.target, test_size=percent_, random_state=0
        )
            
    def __fit(self):
        self.classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
        self.classifier.fit(self.prices_train, self.target_train)
        
    def __predict(self):
        self.future = self.classifier.predict(self.prices_test)
        
    def __str__(self):
        if self.success is True:
            s = f'RELATÓRIO PARA {self.stock_name} DE {self.start_date} ATÉ {self.end_date}\n\n'
            s += classification_report(self.target_test, self.future)
        else:
            s = 'SEM DADOS PARA RELATÓRIO'
        
        return s

In [ ]:
for stock in ['PETR4.SA', 'USIM5.SA', 'BBAS3.SA']:
    stock_pred = StockPredictKNN(stock, 'yahoo', '2015-01-01', '2018-12-31')
    print(stock_pred)